In [1]:
import sys

import holoviews as hv
import numpy as np

from opts import get_opts
from utils import get_data_base_path, make_figure, notebook_config

sys.path.insert(1, "/Users/user/Documents/Git_repos/climate-epidemics")

import climepi  # noqa
from climepi import climdata, epimod

# Configure notebook
notebook_config()

# Get options
options = get_opts()
year_range = options["year_range"]
locations = [options["location_default"]] + options["locations_additional"]
no_locations = len(locations)
epi_model_names = [
    options["epi_model_name_default"],
    options["epi_model_name_additional"],
]
epi_model_species = [
    options["epi_model_species_default"],
    options["epi_model_species_additional"],
]
polyfit_degree = options["polyfit_degree"]
plot_opts_temp = options["plot_opts_temp"]
plot_opts_epi = options["plot_opts_epi"]
figure_dir = options["figure_dir"]

In [3]:
# Get climate data
ds_clim = (
    climdata.get_example_dataset(
        "isimip_cities", data_dir=get_data_base_path() / "isimip_cities"
    )
    .sel(time=slice(str(year_range[0]), str(year_range[1])))
    .sel(location=locations)
)
# Yearly average temperature data
ds_temp_yearly = ds_clim.climepi.yearly_average("temperature")
ds_temp_yearly["time"] = ds_temp_yearly["time.year"]  # avoids plotting issues

In [12]:
# Climate variability - plume
panels_clim_plume = []
for location in locations:
    panels_clim_plume += [
        ds_temp_yearly.sel(location=location)
        .climepi.plot_ci_plume(polyfit_degree=polyfit_degree)
        .opts(
            **{
                **plot_opts_temp,
                "ylim": (9, 24),
                "yticks": np.arange(9, 25, 3),
            }
        )
    ]

In [13]:
# Show panels
hv.Layout(
    [
        p.opts(title=location, show_legend=show_legend)
        for p, location, show_legend in zip(
            panels_clim_plume, locations, [True] + [False] * (2 * no_locations - 1)
        )
    ]
).cols(2)

:Layout
   .Overlay.I   :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean                :Curve   [time]   (Temperature)
   .Overlay.II  :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean                :Curve   [time]   (Temperature)
   .Overlay.III :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean                :Curve   [time]   (Temperature)
   .Overlay.IV  :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean                :Curve   [time]   (Temperature)
   .Overlay.V   :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean                :Curve   [time]   (Temperature)

In [9]:
# Epi variability - plume
panels_epi_plume = []
for epi_model_name, species_curr in zip(epi_model_names, epi_model_species):
    epi_model = epimod.get_example_model(epi_model_name)
    ds_months_suitable = ds_clim.climepi.run_epi_model(
        epi_model, return_months_suitable=True
    )
    ds_months_suitable["time"] = ds_months_suitable["time.year"]
    for location in locations:
        panels_epi_plume += [
            ds_months_suitable.sel(location=location)
            .climepi.plot_ci_plume(polyfit_degree=polyfit_degree)
            .opts(
                **{
                    **plot_opts_epi,
                    "ylim": (0, 12),
                    "yticks": np.arange(0, 13),
                    "ylabel": f"Months suitable ({species_curr})",
                }
            )
        ]

In [10]:
# Show panels
titles = [
    f"{location}, {species}" for species in epi_model_species for location in locations
]
hv.Layout(
    [
        p.opts(title=title, show_legend=show_legend)
        for p, title, show_legend in zip(
            panels_epi_plume, titles, [True] + [False] * (2 * no_locations - 1)
        )
    ]
).cols(2)

:Layout
   .Overlay.I    :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean                :Curve   [time]   (Months where suitability > 0)
   .Overlay.II   :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean                :Curve   [time]   (Months where suitability > 0)
   .Overlay.III  :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean                :Curve   [time]   (Months where suitability > 0)
   .Overlay.IV   :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean                :Curve   [time]   (Months where suitability > 0)
   .Overlay.V    :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean                :Curve   [time]   (Months where suitability > 0)
   .Overlay.VI   :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean                :Curve   [time]   (Months where suitability > 0)
   .Overlay.VII  :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean                :Curve   [time]   (Months where suitability > 0)
   .Overlay.VIII :Overlay
      .Area.Scenario_uncertainty :Area   [time]   (scenario_lower,model_lower)
      .Area.I                    :Area   [time]   (model_upper,scenario_upper)
      .Area.Model_uncertainty    :Area   [time]   (model_lower,internal_lower)
      .Area.II                   :Area   [time]   (internal_upper,model_upper)
      .Area.Internal_variability :Area   [time]   (internal_lower,internal_upper)
      .Curve.Mean         

In [14]:
# Combine panels, ordered by location
panels = [
    p
    for panels_location_curr in zip(
        panels_clim_plume,
        panels_epi_plume[:no_locations],
        panels_epi_plume[no_locations:],
    )
    for p in panels_location_curr
]
make_figure(
    figure_dir / "sensitivity_location_model.svg",
    panels,
    show_legend_list=[True] + [False] * (3 * no_locations - 1),
    reverse_legend_entries_list=[True] + [None] * (3 * no_locations - 1),
)